In [1]:
import pandas as pd
from nested_lookup import nested_lookup
import json
from urllib.parse import urlparse, parse_qs



In [2]:
df = pd.read_csv('./configs/edomizil/destination_ids.csv')
df.to_dict(orient='records')

[{'id': '5460aeabb3b30', ' name': ' Valais'},
 {'id': '5460aeabb3b30', ' name': ' Sorate'}]

In [ ]:
def format_detail_url(url:str) -> str | None:
    base_url = "https://www.e-domizil.ch"

    def url_format_valid() -> bool:
        required_params = [
                            'arrival','clickId','directId',
                            'duration','id','isHotel',
                            'location','pCon','pricetype',
                            'prodName','prodSource','sT',
                            'screen','searchId','timestamp'
                        ]
        url_params = list(parse_qs(urlparse(base_url+ url).query).keys()).sort()

        return required_params == url_params
    
    if url_format_valid(url):
        return base_url + url 
    return None

#url format
id_hotel?
duration&
location&
arrival&
pricetype&
timestamp&
id&
searchId&
screen&
isHotel&
clickId&
sT&pCon&
prodName&
prodSource&
directId"

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.firefox.service import Service as FirefoxService
import pandas as pd 
import re, os, time, csv
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
from random import randint

In [4]:
url = "https://www.e-domizil.ch/rental/004498214a97b35c50da971f2c927a90?duration=7&location=5460aeabb3b30&arrival=2023-10-28&pricetype=totalPrice&timestamp=2023-10-24T14%3A42%3A30%2B02%3A00&id=1945116e20aec4ed&directId=004498214a97b35c50da971f2c927a90&searchId=e3ffefbabcb72866&screen=search&isHotel=0&clickId=3J8ZV2CD46QL5VSH&sT=withDates&pCon=970%7CEUR%7C2023-10-28%7C7%7C2%7C1698151350%7C0%7C0%7C1698151350%7C2%7C0%7C0%7C95%7C1306%7C0&prodName=JM&prodSource=Search"

In [5]:
url2 = "https://www.e-domizil.ch/rental/0be4b0c36369c301b44958e6703238c5?c=EUR&clickId=V6D5QZQY20D62Q1P&hl=fr_CH&isHotel=0&location=5460aeabb3b30&pricetype=totalPrice&prodName=JM&prodSource=Search&sT=withDates&screen=search&searchId=f17cee4c389fa81a&serpAvl=booked&timestamp=2023-10-20T17%3A05%3A48%2B02%3A00&expand=1%2C2%2C3&offers=0be4b0c36369c301b44958e6703238c5&arrival=2023-11-11&duration=7&adults=2&bedrooms=0&pets=0&isExtrasTouched=0&pageType=details"

In [9]:
list(parse_qs(urlparse(url).query).keys())

['duration',
 'location',
 'arrival',
 'pricetype',
 'timestamp',
 'id',
 'directId',
 'searchId',
 'screen',
 'isHotel',
 'clickId',
 'sT',
 'pCon',
 'prodName',
 'prodSource']

In [30]:
list(parse_qs(urlparse(url2).query).keys())

['c',
 'clickId',
 'hl',
 'isHotel',
 'location',
 'pricetype',
 'prodName',
 'prodSource',
 'sT',
 'screen',
 'searchId',
 'serpAvl',
 'timestamp',
 'expand',
 'offers',
 'arrival',
 'duration',
 'adults',
 'bedrooms',
 'pets',
 'isExtrasTouched',
 'pageType']

In [6]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--disable-gpu')
# firefox_options.add_argument('--headless')
chrome_options.add_argument('--incognito')

In [7]:
driver = webdriver.Chrome( options=chrome_options)
driver.maximize_window()

In [8]:
def normalize_url(url:str) -> str:
    url_params = list(parse_qs(urlparse(url).query).keys())
    if 'c' in url_params and 'hl' in url_params:
        return url 
    else:
        if 'c' not in url_params:
            url += '&c=EUR'
        if 'hl' not in url_params:
            url += '&hl=fr_CH'
        return url

In [9]:
normalize_url(url2)

'https://www.e-domizil.ch/rental/0be4b0c36369c301b44958e6703238c5?c=EUR&clickId=V6D5QZQY20D62Q1P&hl=fr_CH&isHotel=0&location=5460aeabb3b30&pricetype=totalPrice&prodName=JM&prodSource=Search&sT=withDates&screen=search&searchId=f17cee4c389fa81a&serpAvl=booked&timestamp=2023-10-20T17%3A05%3A48%2B02%3A00&expand=1%2C2%2C3&offers=0be4b0c36369c301b44958e6703238c5&arrival=2023-11-11&duration=7&adults=2&bedrooms=0&pets=0&isExtrasTouched=0&pageType=details'

In [10]:
driver.get(normalize_url(url2))

In [11]:
def soupify(element:str) -> object:
    from bs4 import BeautifulSoup
    return BeautifulSoup(element, 'lxml')

In [12]:
info_container = driver.find_element(By.XPATH, "//div[@data-test='rental-sidebar']").get_attribute('innerHTML')

In [13]:
info_cleaned = soupify(info_container)

In [14]:
info_cleaned.find('div', {'data-test':"available-badge"})['class']

['df',
 'jcsb',
 'aic',
 'text-medium',
 'ph16',
 'fxg1',
 'rows>m4',
 'bg-success-super-light',
 'c-success-normal',
 'test-available-badge',
 'txt-strong',
 'bdrsm',
 'pt8',
 'pb16']

In [15]:
parse_qs(urlparse(driver.current_url).query)['arrival']

['2023-11-11']

In [16]:
if info_cleaned.find('div', {'data-test':"available-badge"}) and 'bg-success-super-light' in info_cleaned.find('div', {'data-test':"available-badge"})['class']:
    identifiant = info_cleaned.find('div', {'class':"bdtlrsm bdtrrsm bgc-gray-extra-light c-gray-dark pv4 tac text-small"}).find_all('span')[-1].text.strip()
    typologie = info_cleaned.find('div', {'class':"text-overflow text-small txt-strong"}).text.strip()
    location = info_cleaned.find('div', {'class':"rows>m4"}).text.strip().split('personnes')[-1].replace(',', ' -')
    price = info_cleaned.find('div', {'class':"heading-medium"}).text.strip()[1:]
    arrival_date = datetime.strptime(parse_qs(urlparse(driver.current_url).query)['arrival'][0], "%Y-%m-%d").strftime("%d/%m/%Y")
    data = {
        'date_scrap': datetime.now().strftime("%d/%m/%Y"),
        'date_debut': arrival_date,
        'date_fin': (datetime.strptime(arrival_date, "%d/%m/%Y") + timedelta(days=7)).strftime("%d/%m/%Y"),
        'price': price,
        'identifiant': identifiant,
        'typologie': typologie,
        'location': location
    }
    print(data)


{'date_scrap': '03/11/2023', 'date_debut': '11/11/2023', 'date_fin': '18/11/2023', 'price': '678.91', 'identifiant': 'H49W6MH', 'typologie': '30 m² Appartement ∙ 1 chambre ∙ 2 personnes', 'location': 'Loèche-les-Bains - Valais'}


In [6]:
from datetime import datetime, timedelta
monday = datetime.now() - timedelta(days = datetime.now().weekday())
monday.strftime("%d_%m_%Y")

'23_10_2023'

In [ ]:
import json 
item = ""
with open('./static/edomizil/23_10_2023/responses.json', 'r') as openfile:
    content = json.loads(openfile.read())
    print(content)
    

In [7]:
from datetime import datetime, timedelta


datetime.strptime('2023-10-28', "%Y-%m-%d").weekday()

5